In [36]:
# HW#4 Pandas - Academy of Py

In [37]:
#Dependencies
import os
import pandas as pd

In [38]:
#Load 'schools_complete.cSV' file
csvpath = os.path.join('..','raw_data','schools_complete.csv')
schools = pd.read_csv(csvpath)
schools.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [39]:
#Load 'students_complete.csv' file
csvpath2 = os.path.join('..','raw_data','students_complete.csv')
students = pd.read_csv(csvpath2)
students.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [5]:
#Merge the two files into one master data frame.

#Make school name have the same column name in both files.  Differentiate the name columns in the two files.
schools = schools.rename(columns = {'School ID': 'school_id','name': 'school_name'})
students = students.rename(columns = {'school':'school_name', 'name':'student_name','Student ID':'student_id'})
merge_df = pd.merge(schools, students, on = 'school_name')
merge_df.head()

,school_id,school_name,type,size,budget,student_id,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [6]:
# PART 1: District Summary: High level snapshot (in table form) of the district's key metrics, including:
  #Total Schools, Total Students, Total Budget, Average Math Score, Average Reading Score
  #% Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)#Total Schoools

In [7]:
#Total schools and total students
total_schools = merge_df['school_id'].nunique()
print('Total Schools: ', total_schools)

total_students = merge_df['student_id'].nunique()
print('Total Students: ', total_students)

Total Schools:  15
Total Students:  39170


In [8]:
#To confirm that student_ids aren't repeated at more than one school, 
#I compared whether the number of student_id's in merge_df with the number of rows in the original students file.
total_students == students['student_id'].count()

True

In [9]:
#Total Budget: Sum up the budget across all schools, using the original schools dataframe.
total_budget = schools['budget'].sum()
total_budget = '${:,.2f}'.format(total_budget)
print('Total Budget: ', total_budget)

Total Budget:  $24,649,428.00


In [10]:
#Average Math and Reading Scores

avg_math = merge_df['math_score'].mean()
print('Average Math: ',round(avg_math,2))

avg_reading = merge_df['reading_score'].mean()
print('Average Reading: ', round(avg_reading,2))

Average Math:  78.99
Average Reading:  81.88


In [11]:
# % Passing: Math, Reading, and Overall Passing Rate

#First confirm that there are no null score. 
#If there were NaN values, then I'd have to divide by the sum of non-NaN scores rather than by total_students.
merge_df['math_score'].isnull().values.any()

False

In [12]:
#Since there are no null values:
#Math
total_math_passing = sum(merge_df['math_score'] >= 70)
perc_math_passing = total_math_passing/total_students *100
print('% Passing Math: ', perc_math_passing)

#Reading
total_read_passing = sum(merge_df['reading_score'] >= 70)
perc_read_passing = total_read_passing/total_students *100
print('% Passing Reading: ', perc_read_passing)

% Passing Math:  74.9808526934
% Passing Reading:  85.8054633648


In [13]:
#Create an Overall Score: Average of Math and Reading
merge_df['overall_score'] = (merge_df['math_score'] + merge_df['reading_score'])/2
merge_df.head()

,school_id,school_name,type,size,budget,student_id,student_name,gender,grade,reading_score,math_score,overall_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79,72.5
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61,77.5
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60,75.0
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58,62.5
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84,90.5


In [14]:
total_overall_passing = sum(merge_df['overall_score'] > 70)
perc_overall_passing = total_overall_passing/total_students *100
print('% Passing Overall: ', perc_overall_passing)

% Passing Overall:  88.2333418432


In [15]:
#District Summary Table: Putting it all together
district_summary = {'Total Schools': total_schools,
                   'Total Students': total_students,
                   'Total Budget': total_budget,
                   'Average Math Score': avg_math,
                   'Average Reading Score': avg_reading,
                   '% Passing Math': perc_math_passing,
                   '% Passing Reading': perc_read_passing,
                   '% Overall Passing Rate': perc_overall_passing}
district_summary

{'% Overall Passing Rate': 88.233341843247388,
 '% Passing Math': 74.980852693387803,
 '% Passing Reading': 85.805463364820014,
 'Average Math Score': 78.98537145774827,
 'Average Reading Score': 81.87784018381414,
 'Total Budget': '$24,649,428.00',
 'Total Schools': 15,
 'Total Students': 39170}

In [16]:
# Part 2: School Summary: Overview table that summarizes key metrics about each school, including:
# School Name, School Type, Total Students, Total School Budget, Per School Budget, Average Math Score, Average Reading Score
# % Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)

In [17]:
def passing(score):
    if score >= 70:
        return 1
    else:
        return 0

merge_df['reading_pass'] = merge_df['reading_score'].apply(passing)
merge_df['math_pass'] = merge_df['math_score'].apply(passing)
merge_df['overall_pass'] = merge_df['overall_score'].apply(passing)
merge_df.head()

,school_id,school_name,type,size,budget,student_id,student_name,gender,grade,reading_score,math_score,overall_score,reading_pass,math_pass,overall_pass
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79,72.5,0,1,1
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61,77.5,1,0,1
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60,75.0,1,0,1
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58,62.5,0,0,0
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84,90.5,1,1,1


In [18]:
#Create master grouped dataframe
school_group = merge_df.groupby('school_name')

In [19]:
#Calculate count of total_students on school_group
grouped_total_students = pd.DataFrame(school_group['student_id'].count())
grouped_total_students = grouped_total_students.reset_index()
grouped_total_students = grouped_total_students.rename(columns = {'student_id': 'Total Students'})
grouped_total_students.head()

,school_name,Total Students
0,Bailey High School,4976
1,Cabrera High School,1858
2,Figueroa High School,2949
3,Ford High School,2739
4,Griffin High School,1468


In [20]:
#Calculate sum of reading_pass, math_pass on school_group
grouped_total_pass = pd.DataFrame(school_group['reading_pass','math_pass','overall_pass'].sum())
grouped_total_pass = grouped_total_pass.reset_index()
grouped_total_pass.head()

,school_name,reading_pass,math_pass,overall_pass
0,Bailey High School,4077,3318,4239
1,Cabrera High School,1803,1749,1850
2,Figueroa High School,2381,1946,2497
3,Ford High School,2172,1871,2322
4,Griffin High School,1426,1371,1460


In [21]:
#Calculate avg math and reading scores on school_group
grouped_avg_math_read = school_group['math_score','reading_score'].mean()
grouped_avg_math_read = grouped_avg_math_read.reset_index()
grouped_avg_math_read = grouped_avg_math_read.rename(columns = {'math_score':'Average Math Score',
                                                               'reading_score': 'Average Reading Score'})
grouped_avg_math_read.head()

,school_name,Average Math Score,Average Reading Score
0,Bailey High School,77.048432,81.033963
1,Cabrera High School,83.061895,83.975780
2,Figueroa High School,76.711767,81.158020
3,Ford High School,77.102592,80.746258
4,Griffin High School,83.351499,83.816757


In [22]:
#Construct Data Frame of the grouped 

#When I tried to merge multiple data sets in one step, it didn't work:
#school_summary = pd.merge(schools,grouped_total_students,grouped_total_pass,grouped_avg_math_read,on = 'school_name')
#So I merged two tables at a time:

merge1 = pd.merge(schools,grouped_total_students, on = 'school_name')
merge2 = pd.merge(merge1, grouped_total_pass, on='school_name')
merge3 = pd.merge(merge2, grouped_avg_math_read, on = 'school_name')
merge3.head()

,school_id,school_name,type,size,budget,Total Students,reading_pass,math_pass,overall_pass,Average Math Score,Average Reading Score
0,0,Huang High School,District,2917,1910635,2917,2372,1916,2479,76.629414,81.182722
1,1,Figueroa High School,District,2949,1884411,2949,2381,1946,2497,76.711767,81.158020
2,2,Shelton High School,Charter,1761,1056600,1761,1688,1653,1750,83.359455,83.725724
3,3,Hernandez High School,District,4635,3022020,4635,3748,3094,3934,77.289752,80.934412
4,4,Griffin High School,Charter,1468,917500,1468,1426,1371,1460,83.351499,83.816757


In [23]:
#Calulate Per Student Budget
merge3['Per Student Budget'] = merge3['budget']/merge3['Total Students']
merge3['Per Student Budget'] = merge3['Per Student Budget'].map('${:,.2f}'.format)

In [24]:
#Calculate Passing Percentages
merge3['% Passing Math'] = merge3['math_pass']/merge3['Total Students']*100
merge3['% Passing Reading'] = merge3['reading_pass']/merge3['Total Students']*100
merge3['% Overall Passing Rate'] = merge3['overall_pass']/merge3['Total Students']*100

In [25]:
#Drop columns...size is redundant
merge4 = merge3.drop(['school_id','size', 'reading_pass', 'math_pass','overall_pass'], axis = 1)

#Rename columns
merge4 = merge4.rename(columns = {'school_name': "School Name",
                                 'type': 'School Type',
                                 'budget': 'Total School Budget'})
#Rearrange column order
merge4 = merge4[['School Name','School Type','Total Students','Total School Budget','Per Student Budget',
                'Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]

#Format Total School Budget
merge4['Total School Budget'] = merge4['Total School Budget'].map("${:,.2f}".format)

merge4

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,84.984573
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,84.672770
2,Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,99.375355
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,84.875944
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,99.455041
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,99.255366
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,99.569429
7,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,85.188907
8,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,98.594848
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,99.168399


In [26]:
#Part 3
#Top Performing Schools (By Passing Rate): Table of top 5 performing schools based on Overall Passing Rate. 

# Sort by % Overall Passing Rate column to find the best
merge4 = merge4.sort_values(by='% Overall Passing Rate', ascending=False)

In [27]:
#Top 5 Performing Schools
top_five = merge4.iloc[:5]
top_five

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,99.569429
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,99.455041
2,Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,99.375355
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,99.255366
10,Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,93.333333,96.611111,99.222222


In [28]:
#Bottom 5 Peforming Schools

bottom_five = merge4.iloc[-5:]
bottom_five

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
12,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,84.982147
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,84.875944
13,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,84.775465
11,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,84.746187
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,84.672770


In [29]:
#Part 4: Math Scores by Grade: Table that lists the average Math Score for students of each grade level at each school.

grade_group = merge_df.groupby(['school_name','grade'])
grade_math = grade_group['math_score'].mean()
grade_math = pd.DataFrame(grade_math)
grade_math = grade_math.reset_index()
grade_math = grade_math.rename(columns={'math_score':'Average Math Score'})
grade_math.head(10)

,school_name,grade,Average Math Score
0,Bailey High School,10th,76.996772
1,Bailey High School,11th,77.515588
2,Bailey High School,12th,76.492218
3,Bailey High School,9th,77.083676
4,Cabrera High School,10th,83.154506
5,Cabrera High School,11th,82.765560
6,Cabrera High School,12th,83.277487
7,Cabrera High School,9th,83.094697
8,Figueroa High School,10th,76.539974
9,Figueroa High School,11th,76.884344


In [30]:
#Part 5: Reading Scores by Grade
# Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

grade_group = merge_df.groupby(['school_name','grade'])
grade_reading = grade_group['reading_score'].mean()
grade_reading = pd.DataFrame(grade_reading)
grade_reading = grade_reading.reset_index()
grade_reading = grade_reading.rename(columns={'reading_score':'Average Reading Score'})
grade_reading.head(10)

,school_name,grade,Average Reading Score
0,Bailey High School,10th,80.907183
1,Bailey High School,11th,80.945643
2,Bailey High School,12th,80.912451
3,Bailey High School,9th,81.303155
4,Cabrera High School,10th,84.253219
5,Cabrera High School,11th,83.788382
6,Cabrera High School,12th,84.287958
7,Cabrera High School,9th,83.676136
8,Figueroa High School,10th,81.408912
9,Figueroa High School,11th,80.640339


In [31]:
#Part 6: Scores by School Spending
#Create a table that breaks down school performances based on average Spending Ranges (Per Student). 
#Use 4 reasonable bins to group school spending. Include in the table each of the following:
#Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, Overall Passing Rate (Average of the above two)

#Convert the Per Student Budget column back to float
merge4['Per Student Budget'] = merge4['Per Student Budget'].replace('[\$,]','',regex=True).astype(float)
print(type(merge4['Per Student Budget'][0]))

<class 'numpy.float64'>


In [32]:
#Create the bins and group by
bins = [0,585,615,645,675]
group_names = ['<$585', '$585-615','$615-645','$645-675']
merge4['School Spend Perf'] = pd.cut(merge4['Per Student Budget'], bins, labels=group_names)
merge4_group = merge4.groupby('School Spend Perf').mean()

#Drop unnecessary columns
columns_to_drop = ['Total Students','Per Student Budget']
merge4_group.drop(columns_to_drop, axis=1, inplace = True)
merge4_group.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Spend Perf,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,99.160466
$585-615,83.599686,83.885211,94.230858,95.900287,99.271877
$615-645,79.079225,81.891436,75.668212,86.106569,89.653490
$645-675,76.997210,81.027843,66.164813,81.133951,84.947555


In [33]:
#Part 7: Scores by School Size
#Repeat the above breakdown, 
#but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).

#Dviide the range evenly into thirds to get 3 bins
range = (merge4['Total Students'].max()) - (merge4['Total Students'].min())
bin_size = int(range/3)
bin_1_max = merge4['Total Students'].min() + bin_size
bin_2_max = bin_1_max + bin_size

bins = [merge4['Total Students'].min(), bin_1_max, bin_2_max, merge4['Total Students'].max()]
print(bins)


[427, 1943, 3459, 4976]


In [34]:
group_names = ['Small','Medium', 'Large']

merge4['School Size'] = pd.cut(merge4['Total Students'], bins, labels = group_names)
merge4_size_group = merge4.groupby('School Size').mean()
merge4_size_group.drop(columns_to_drop, axis=1, inplace = True)
merge4_size_group.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small,83.452223,83.894482,93.765511,96.649891,99.312169
Medium,78.429493,81.769122,73.462428,84.473577,88.422044
Large,77.063340,80.919864,66.464293,81.059691,84.948296


In [35]:
#Part 8: Scores by School Type
#Repeat the above breakdown, 
#but this time group schools based on school type (Charter vs. District)

merge4_type_group = merge4.groupby('School Type').mean()
merge4_type_group.drop(columns_to_drop, axis = 1, inplace = True)
merge4_type_group.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,99.215404
District,76.956733,80.966636,66.548453,80.799062,84.889428


In [ ]:
#Three Observable Trends

#1. Suprisingly, school budgets over $615 per student had a ten percentage point lower overall passing rate.
#than school budgets less than $615 per student.

#2.  As I would have guessed, there seems to be a tsrend of overall passing rates increasing with smaller school size.

#3. Charter schools have better student performance than district schools.

#4. One thing to consider are confounding factors such as family income level of student.
#If this was controlled for, I wonder if we would see a statistically significant impact of
#school budget, size, and school typ on student performance.
